## Modifying Parameters in System

Some machine learning force field construction schemes, such as [Espaloma](https://pubs.rsc.org/en/content/articlehtml/2022/sc/d2sc02739a), would output parameters which can be picked up by the parameter-holding `System` object of OpenMM, which could be further used in molecular dynamics (MD) simulations just like a vanilla `System`.
In this tutorial, we show how you can modify the parameters of `System` object so you can write your own machine learning force field API.

This example is inspired by the [Espaloma deployment](https://github.com/choderalab/espaloma/blob/master/espaloma/graphs/deploy.py) implementation.

First, let's load a PDB file and use Amber14 force field as placeholder parameters.

In [1]:
from openmm.app import *
from openmm import *

pdb = PDBFile('ala_ala_ala.pdb')
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
system = forcefield.createSystem(pdb.topology)

Now we loop through the forces to get bonded components of the energy.

In [6]:
forces = list(system.getForces())

In [7]:
harmonic_bond_force = [force for force in forces if "HarmonicBondForce" in force.__class__.__name__].pop()
harmonic_angle_force = [force for force in forces if "HarmonicAngleForce" in force.__class__.__name__].pop()
periodic_torsion_force = [force for force in forces if "PeriodicTorsionForce" in force.__class__.__name__].pop()

We can inspect each component to get a list of indices and parameters.

- Bonds:

In [9]:
for idx in range(harmonic_bond_force.getNumBonds()):
    idx0, idx1, r0, k = harmonic_bond_force.getBondParameters(idx)
    print(f"Bond between {idx0} and {idx1} has equilibrium length {r0} and force constant {k}.")

Bond between 10 and 4 has equilibrium length 0.1522 nm and force constant 265265.6 kJ/(nm**2 mol).
Bond between 10 and 11 has equilibrium length 0.12290000000000001 nm and force constant 476975.9999999999 kJ/(nm**2 mol).
Bond between 4 and 6 has equilibrium length 0.1526 nm and force constant 259407.99999999994 kJ/(nm**2 mol).
Bond between 4 and 5 has equilibrium length 0.10900000000000001 nm and force constant 284511.99999999994 kJ/(nm**2 mol).
Bond between 4 and 0 has equilibrium length 0.1471 nm and force constant 307105.5999999999 kJ/(nm**2 mol).
Bond between 6 and 7 has equilibrium length 0.10900000000000001 nm and force constant 284511.99999999994 kJ/(nm**2 mol).
Bond between 6 and 8 has equilibrium length 0.10900000000000001 nm and force constant 284511.99999999994 kJ/(nm**2 mol).
Bond between 6 and 9 has equilibrium length 0.10900000000000001 nm and force constant 284511.99999999994 kJ/(nm**2 mol).
Bond between 1 and 0 has equilibrium length 0.101 nm and force constant 363171.1

- angles:

In [10]:
for idx in range(harmonic_angle_force.getNumAngles()):
    idx0, idx1, idx2, r0, k = harmonic_angle_force.getAngleParameters(idx)
    print(f"Angle among {idx0}, {idx1}, and {idx2} has equilibrium angle {r0} and force constant {k}.")

Angle among 0, 4, and 5 has equilibrium angle 1.911135530933791 rad and force constant 418.40000000000003 kJ/(mol rad**2).
Angle among 0, 4, and 6 has equilibrium angle 1.9408061282176945 rad and force constant 669.44 kJ/(mol rad**2).
Angle among 0, 4, and 10 has equilibrium angle 1.9408061282176945 rad and force constant 669.44 kJ/(mol rad**2).
Angle among 1, 0, and 2 has equilibrium angle 1.911135530933791 rad and force constant 292.88 kJ/(mol rad**2).
Angle among 1, 0, and 3 has equilibrium angle 1.911135530933791 rad and force constant 292.88 kJ/(mol rad**2).
Angle among 1, 0, and 4 has equilibrium angle 1.911135530933791 rad and force constant 418.40000000000003 kJ/(mol rad**2).
Angle among 2, 0, and 3 has equilibrium angle 1.911135530933791 rad and force constant 292.88 kJ/(mol rad**2).
Angle among 2, 0, and 4 has equilibrium angle 1.911135530933791 rad and force constant 418.40000000000003 kJ/(mol rad**2).
Angle among 3, 0, and 4 has equilibrium angle 1.911135530933791 rad and f

Interestingly, one can also modify such parameters in-place.
In a machine learning framework, usually such modification is done gloabally.
We here only show how to modify a few bonds and angles, whereas global re-parametrization can be easily achieved in a similar manner with a larger `PyTree` holding all parameters.

Say we would want to modify the bond parameters of the bond between atom 0 and 1.

In [27]:
for idx in range(harmonic_bond_force.getNumBonds()):
    idx0, idx1, r0, k = harmonic_bond_force.getBondParameters(idx)
    if idx0 == 1 and idx1 == 0:
        new_r0 = unit.Quantity(2666, unit.nanometer)
        new_k = unit.Quantity(1984, unit.kilojoule / (unit.nanometer ** 2 * unit.mole))
        harmonic_bond_force.setBondParameters(idx, idx0, idx1, new_r0, new_k)

Now if you query again you will see that the bond parameter has been changed.

Similarly, for angles:

In [34]:
for idx in range(harmonic_angle_force.getNumAngles()):
    idx0, idx1, idx2, r0, k = harmonic_angle_force.getAngleParameters(idx)
    if idx0 == 1 and idx1 == 0 and idx2 == 2:        
        new_r0 = unit.Quantity(2666, unit.radian)
        new_k = unit.Quantity(1984, unit.kilojoule / (unit.radian ** 2 * unit.mole))
        harmonic_angle_force.setAngleParameters(idx, idx0, idx1, idx2, new_r0, new_k)

Note that we can use the `unit` module to control the units of the parameters.